# Pixel Drill for Indices outputs to CSV - Multi-site

| Authors:  | Bex Dunn|
|----------|----------------|
| Created: | March 6, 2019 |
| Last edited: | Nov 12, 2019 |

**Requirements:**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles` 

`module load dea`

If you find an error or bug in this notebook, please either create an 'Issue' in the Github repository, or fix it yourself and create a 'Pull' request to contribute the updated notebook back into the repository (See the repository [README](https://github.com/GeoscienceAustralia/dea-notebooks/blob/master/README.rst) for instructions on creating a Pull request).

__Background:__ Data from the [Landsat](https://landsat.usgs.gov/about-landsat) 5,7 and 8 satellite missions are accessible through [Digital Earth Australia](http://www.ga.gov.au/about/projects/geographic/digital-earth-australia) (DEA).

__What does this notebook do?:__ This notebook takes a supplied CSV of site points. It runs a pixel drill through surface reflectance, calculates NDVI, Taselled cap wetness and greenness, and outputs a csv of values for each site and plots of each index for each site.

**Tags**: :index:`Landsat`,:index:`Landsat5`,:index:`Landsat7`,:index:`Landsat8`, :index:`pixeldrill`, :index:`DEAPlotting`, :index:`datacube.utils.geometry`, :index:`query`,:index:`Scripts`,:index:`tasseled_cap`, :index:`NDVI`,                                                                                                           :index:`DEADataHandling`, :index:`DEAPlotting`, :index:`load_clearlandsat`

import some modules

In [5]:
import datacube

In [6]:
import datacube
import datetime
import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
import shapely
import seaborn as sns
import sys
import xarray as xr

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from datacube.storage import masking
from datacube.utils import geometry

sys.path.append('../../10_Scripts')
import DEADataHandling, DEAPlotting, TasseledCapTools, BandIndices

dc = datacube.Datacube(app='pixel drill')

%load_ext autoreload

%autoreload 2

#set up file to open 

In [7]:
inpath = '/g/data/r78/rjd547/Melbourne_water/all_sites.csv'
### Convert csv latitude and longitude values into a geopandas geodatafrome

sites = pd.read_csv(inpath, delimiter=",")
#turn csv geometry into geopandas dataframe geometry using lambda functions and shapely

In [10]:
sites['y']=sites['lat']
sites['x']=sites['lng']

In [11]:
sites['geometry']=sites.apply(lambda z: shapely.geometry.Point(z.x, z.y), axis=1)
sites = gpd.GeoDataFrame(sites)
sites.head()

#define an output location
output_loc = '/g/data/r78/rjd547/Melbourne_water/all_sites_1119/point_'

### run this for multiple sites

for site in range(0,len(sites)):    
    print(f'running for index {site}')
    lon = sites['x'][site]
    lat = sites['y'][site]
    query = {'lat':lat, 
          'lon':lon}               
    #print(query)
    outfilename=f'{output_loc}{lat}_{lon}.csv'
    print (outfilename)

    try:
        ls578 = DEADataHandling.load_clearlandsat(dc=dc, query=query, product='nbart', ls7_slc_off=True)
    except:
        print ('nah')
        ### Calculate NDVI 

    ### Calculate the tasseled cap indices
    tci = TasseledCapTools.thresholded_tasseled_cap(ls578,wetness_threshold=-350, drop=True , drop_tc_bands=False)
    
    #drop thresholded bands; our stakeholder wants these unthresholded
    tci = tci.drop(['greenness_thresholded','brightness_thresholded','wetness_thresholded'])

    #drop the 1-dimensional x and ys
    tci =tci.squeeze()

    #make a dataframe for writing to csv
    tci_df = tci.to_dataframe()
    tci_df = tci_df.drop(columns=['y','x'])
    #drop all nan-slices
    tci_df = tci_df.dropna(axis='index', thresh =2)

    #write our csv
    tci_df.to_csv(outfilename)

    !ls /g/data/r78/rjd547/Melbourne_water/more_points/*.csv     

running for index 0
/g/data/r78/rjd547/Melbourne_water/all_sites_1119/point_-37.23055489_144.78120180000002.csv
Loading ls5
    Loading 314 filtered ls5 timesteps
Loading ls7
    Loading 305 filtered ls7 timesteps
Loading ls8
    Loading 148 filtered ls8 timesteps
Combining and sorting ls5, ls7, ls8 data
    Replacing invalid -999 values with NaN (data will be coerced to float64)
/g/data/r78/rjd547/Melbourne_water/more_points/point_-38.11110516_145.8223497.csv
/g/data/r78/rjd547/Melbourne_water/more_points/point_-38.13884572_145.8292751.csv
/g/data/r78/rjd547/Melbourne_water/more_points/point_-38.14588293_145.6080329.csv
/g/data/r78/rjd547/Melbourne_water/more_points/point_-38.14700344_145.6840971.csv
/g/data/r78/rjd547/Melbourne_water/more_points/point_-38.15054097_145.3881909.csv
/g/data/r78/rjd547/Melbourne_water/more_points/point_-38.15116122_145.5127214.csv
/g/data/r78/rjd547/Melbourne_water/more_points/point_-38.1700706_145.1863836.csv
/g/data/r78/rjd547/Melbourne_water/more_poin

In [2]:
!ls /g/data/r78/rjd547/Melbourne_water/all_sites_1119/